**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Start

Deployment and Service

https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/

In [3]:
kubectl get pods -A

NAMESPACE     NAME                               READY   STATUS    RESTARTS        AGE
kube-system   coredns-78fcd69978-v78km           1/1     Running   1 (6m31s ago)   12d
kube-system   etcd-minikube                      1/1     Running   1 (6m31s ago)   12d
kube-system   kube-apiserver-minikube            1/1     Running   1 (6m31s ago)   12d
kube-system   kube-controller-manager-minikube   1/1     Running   1 (6m31s ago)   12d
kube-system   kube-proxy-55mgp                   1/1     Running   1 (6m31s ago)   12d
kube-system   kube-scheduler-minikube            1/1     Running   1 (6m31s ago)   12d
kube-system   storage-provisioner                1/1     Running   3 (5m39s ago)   12d


# Warning

**we install software from an unknown source**

In [4]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

deployment.apps/my-nginx created


In [5]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

service/my-nginx created


In [8]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/my-nginx-5b56ccd65f-28ksz   1/1     Running   0          95s
pod/my-nginx-5b56ccd65f-588gw   1/1     Running   0          95s

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP   12d
service/my-nginx     ClusterIP   10.111.88.126   <none>        80/TCP    85s

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-nginx   2/2     2            2           96s

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/my-nginx-5b56ccd65f   2         2         2       95s


In [9]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

apiVersion: v1
kind: Service
metadata:
  name: my-nginx
  labels:
    run: my-nginx
spec:
  ports:
  - port: 80
    protocol: TCP
  selector:
    run: my-nginx


In [10]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nginx
spec:
  selector:
    matchLabels:
      run: my-nginx
  replicas: 2
  template:
    metadata:
      labels:
        run: my-nginx
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80



In [11]:
kubectl get service my-nginx

NAME       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
my-nginx   ClusterIP   10.111.88.126   <none>        80/TCP    2m41s


In [12]:
kubectl get service my-nginx -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2021-11-24T10:08:36Z"
  labels:
    run: my-nginx
  name: my-nginx
  namespace: default
  resourceVersion: "342232"
  uid: 294a535d-3ec5-4b9d-8cce-6e0dbc3ea064
spec:
  clusterIP: 10.111.88.126
  clusterIPs:
  - 10.111.88.126
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    run: my-nginx
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [13]:
kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}'

10.111.88.126

In [14]:
IP=$(kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}')
curl $IP

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [15]:
echo http://$IP

http://10.111.88.126


In [16]:
kubectl get service -A -o jsonpath='{.items[*].spec.clusterIP}'

10.96.0.1 10.111.88.126 10.96.0.10

In [18]:
kubectl get pods --all-namespaces -o go-template \
--template="{{range .items}}{{.metadata.namespace}}/{{.metadata.name}}:{{println}}{{range .spec.containers}}  {{.image}}:{{.securityContext}}
{{end}}{{end}}" | grep --color=always -E '(^|privileged|NET_BIND_SERVICE)'


default/my-nginx-5b56ccd65f-28ksz:
  nginx:<no value>
default/my-nginx-5b56ccd65f-588gw:
  nginx:<no value>
kube-system/coredns-78fcd69978-v78km:
  k8s.gcr.io/coredns/coredns:v1.8.4:map[allowPrivilegeEscalation:false capabilities:map[add:[NET_BIND_SERVICE] drop:[all]] readOnlyRootFilesystem:true]
kube-system/etcd-minikube:
  k8s.gcr.io/etcd:3.5.0-0:<no value>
kube-system/kube-apiserver-minikube:
  k8s.gcr.io/kube-apiserver:v1.22.2:<no value>
kube-system/kube-controller-manager-minikube:
  k8s.gcr.io/kube-controller-manager:v1.22.2:<no value>
kube-system/kube-proxy-55mgp:
  k8s.gcr.io/kube-proxy:v1.22.2:map[privileged:true]
kube-system/kube-scheduler-minikube:
  k8s.gcr.io/kube-scheduler:v1.22.2:<no value>
kube-system/storage-provisioner:
  gcr.io/k8s-minikube/storage-provisioner:v5:<no value>
